In [1]:
import os 

In [2]:
%pwd

'c:\\Users\\asus\\OneDrive\\Desktop\\Pandas app\\End-to-end-ML-Project\\research'

In [3]:
os.chdir("../")

In [4]:
%pwd

'c:\\Users\\asus\\OneDrive\\Desktop\\Pandas app\\End-to-end-ML-Project'

In [5]:
from dataclasses import dataclass
from pathlib import Path


@dataclass(frozen=True)
class DataValidationConfig:
    root_dir: Path
    status_file: str
    unzip_data_dir: Path
    all_schema: dict

In [6]:
from src.mlProject.constants import *
from src.mlProject.utils.common import read_yaml,create_directories

In [28]:
class ConfigurationManager:
    def __init__(
        self,
        config_filepath = CONFIG_FILE_PATH,
        params_filepath = PARAMS_FILE_PATH,
        schema_filepath = SCHEMA_FILE_PATH):

        self.config = read_yaml(config_filepath)
        self.params = read_yaml(params_filepath)
        self.schema = read_yaml(schema_filepath)

        create_directories([self.config.artifacts_root])


    def get_data_Validation_config(self) -> DataValidationConfig:
        config = self.config.data_validation
        schema=self.schema.COLUMNS
        create_directories([config.root_dir])

        data_validation_config = DataValidationConfig(
            root_dir=config.root_dir,
            status_file=config.STATUS_FILE,
            unzip_data_dir=config.unzip_data_dir,
            all_schema=schema
        )

        return data_validation_config

In [29]:
import os
from mlProject import logger
import pandas as pd

In [43]:
class DataValidation:
    def __init__(self, config: DataValidationConfig):
        self.config=config
    def validate_all_cols(self)->bool:
        try:
            validation_status=None
            data=pd.read_csv(self.config.unzip_data_dir)
            all_cols=list(data.columns)
            all_schema=list(self.config.all_schema.keys())
            for col in all_cols:
                if col not in all_schema:
                    validation_status=False
                    with open(self.config.status_file,'w') as file:
                        file.write(f"validation status is: {validation_status}")
                else:
                    validation_status=True
                    with open(self.config.status_file,'w') as file:
                        file.write(f"validation status is: {validation_status}")
            return validation_status
        except Exception as e:
            raise e


In [44]:
try:
    config = ConfigurationManager()
    data_validation_config = config.get_data_Validation_config()
    data_validation = DataValidation(config=data_validation_config)
    data_validation.validate_all_cols()
except Exception as e:
    raise e

2024-09-06 14:11:10,190 : INFO : common : yaml file: config\config.yaml loaded successfully
2024-09-06 14:11:10,190 : INFO : common : yaml file: config\config.yaml loaded successfully
2024-09-06 14:11:10,192 : INFO : common : yaml file: params.yaml loaded successfully
2024-09-06 14:11:10,192 : INFO : common : yaml file: params.yaml loaded successfully
2024-09-06 14:11:10,199 : INFO : common : yaml file: schema.yaml loaded successfully
2024-09-06 14:11:10,199 : INFO : common : yaml file: schema.yaml loaded successfully
2024-09-06 14:11:10,206 : INFO : common : created directory at: artifacts
2024-09-06 14:11:10,206 : INFO : common : created directory at: artifacts
2024-09-06 14:11:10,209 : INFO : common : created directory at: artifacts/data_validation
2024-09-06 14:11:10,209 : INFO : common : created directory at: artifacts/data_validation


In [41]:
def my_method():
    config_1 = ConfigurationManager()
    new_obj = config_1.get_data_Validation_config()
    print(list(new_obj.all_schema.keys()))
    data=pd.read_csv(new_obj.unzip_data_dir)
    all_cols=list(data.columns)
    print(all_cols)

In [42]:
my_method()

2024-09-06 14:05:45,952 : INFO : common : yaml file: config\config.yaml loaded successfully
2024-09-06 14:05:45,952 : INFO : common : yaml file: config\config.yaml loaded successfully
2024-09-06 14:05:45,957 : INFO : common : yaml file: params.yaml loaded successfully
2024-09-06 14:05:45,957 : INFO : common : yaml file: params.yaml loaded successfully
2024-09-06 14:05:45,962 : INFO : common : yaml file: schema.yaml loaded successfully
2024-09-06 14:05:45,962 : INFO : common : yaml file: schema.yaml loaded successfully
2024-09-06 14:05:45,966 : INFO : common : created directory at: artifacts
2024-09-06 14:05:45,966 : INFO : common : created directory at: artifacts
2024-09-06 14:05:45,970 : INFO : common : created directory at: artifacts/data_validation
2024-09-06 14:05:45,970 : INFO : common : created directory at: artifacts/data_validation
['fixed acidity', 'volatile acidity', 'citric acid', 'residual sugar', 'chlorides', 'free sulfur dioxide', 'total sulfur dioxide', 'density', 'pH', 